In [14]:
import re

import bokeh
import gensim
from gensim.models.word2vec import Word2Vec
import numpy as np
from nltk.tokenize import TweetTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook, tqdm

In [36]:
nlp = spacy.load("en")
tqdm_notebook().pandas(desc="progress-bar")
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [16]:
def load_tweets():
    tweets_train = pd.read_csv("/home/malits/data/twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv", 
                               encoding = "ISO-8859-1")
    data = pd.DataFrame()
    data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v > 2) else 0)
    data["sentiment"] = data["sentiment"].map(int)
    data["tweet"] = tweets_train.iloc[:, [5]]
    data["tweet"] = data["tweet"].astype(str)
    data = data[data['tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    
    return data

data = load_tweets()
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [28]:
def tokenize(tweet):
    tweet = str(tweet)
    tokens = tokenizer.tokenize(tweet)
    tokens = [re.sub(r'#[A-Za-z0-9]+', '', t) for t in tokens]
    tokens = [re.sub(r'https.+', '', t) for t in tokens]
    tokens = [t for t in tokens if t is not None and t != '']
    return tokens

In [32]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data["tokens"] = data["tweet"].progress_apply(tokenize)
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [33]:
data = postprocess(data)

### Create training data

Once preprocessed, split to test/train sets and vectorize w w2vec

In [40]:
N_DIM = 200

In [35]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data.tokens),
                                                    np.array(data.sentiment),
                                                    test_size=0.2)

In [37]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i, v in tqdm_notebook(enumerate(tweets)):
        label = '{}_{}'.format(label_type, i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

In [38]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')

/home/malits/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [ ]:
train_size = len(X_train)
tweet_w2v = Word2Vec(size=N_DIM, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm_notebook(X_train)])
tweet_w2v.train([x.words for x in tqdm_notebook(X_train)], 
                total_examples=train_size, epochs = tweet_w2v.epochs)